In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import GRU, Dense, Dropout
from tensorflow.keras.optimizers import Adam

In [2]:
df= pd.read_csv('GRU-data.csv')
df.head()

,Unnamed: 0.1,날짜,Unnamed: 0,kospi200,kosdaq,dow,nikkei,ssec,usd_krw,cny_krw,jpy_krw,콜금리,국고채,wti,brent,gold,vix,Target
0,0,2017-04-17,1738,278.10,629.47,"20,636.92","18,355.26","3,222.17","1,133.02",164.63,10.4037,1.23,3.611,52.73,55.36,"1,291.90",14.66,1
1,1,2017-04-18,1737,278.23,634.73,"20,636.92","18,418.59","3,196.71","1,133.02",166.03,10.5386,1.23,3.611,52.73,55.36,"1,291.90",14.66,0
2,2,2017-04-19,1736,276.49,635.99,"20,523.28","18,432.20","3,170.69","1,142.64",166.05,10.5056,1.23,3.611,52.33,54.89,"1,294.10",14.42,1
3,3,2017-04-20,1735,277.76,635.80,"20,404.49","18,430.49","3,172.10","1,143.69",165.37,10.4108,1.24,3.611,50.62,52.93,"1,283.40",14.93,1
4,4,2017-04-21,1734,280.05,634.96,"20,578.71","18,620.75","3,173.15","1,138.12",164.77,10.4008,1.24,3.611,50.27,52.99,"1,283.80",14.15,1


In [3]:
df_fi=df.drop(['Unnamed: 0.1','Unnamed: 0','usd_krw', 'cny_krw','gold','wti','jpy_krw','vix', '국고채'], axis=1)
df_fi.head()

,날짜,kospi200,kosdaq,dow,nikkei,ssec,콜금리,brent,Target
0,2017-04-17,278.10,629.47,"20,636.92","18,355.26","3,222.17",1.23,55.36,1
1,2017-04-18,278.23,634.73,"20,636.92","18,418.59","3,196.71",1.23,55.36,0
2,2017-04-19,276.49,635.99,"20,523.28","18,432.20","3,170.69",1.23,54.89,1
3,2017-04-20,277.76,635.80,"20,404.49","18,430.49","3,172.10",1.24,52.93,1
4,2017-04-21,280.05,634.96,"20,578.71","18,620.75","3,173.15",1.24,52.99,1


In [4]:
# 데이터를 역순으로 정렬하여 전체 데이터의 70% 학습, 30% 테스트에 사용
df_fi = df_fi[::-1]
df_fi.head()

,날짜,kospi200,kosdaq,dow,nikkei,ssec,콜금리,brent,Target
1738,2024-05-14,371.04,862.22,"39,431.64","38,314.50","3,146.29",4.917388,83.36,0
1737,2024-05-13,370.98,854.43,"39,431.64","38,179.46","3,148.17",4.910239,83.36,1
1736,2024-05-10,371.08,864.16,"39,387.76","38,257.00","3,155.65",4.888839,83.88,0
1735,2024-05-09,368.84,870.15,"39,055.73","38,070.50","3,155.43",4.881722,83.58,1
1734,2024-05-08,374.09,872.47,"38,883.94","38,202.37","3,128.48",4.874613,82.99,0


In [5]:
# 쉼표 제거 및 숫자 형식으로 변환, 날짜 열은 제외
for column in df.columns:
    if column != '날짜':
        df[column] = df[column].apply(lambda x: str(x).replace(',', '') if isinstance(x, str) else x)
        df[column] = df[column].astype(float)


In [6]:
# 피처와 타겟을 분리합니다
features = df.drop(['Target', '날짜'], axis=1)
target = df['Target']

In [7]:
# 피처 스케일링
scaler = MinMaxScaler()
scaled_features = scaler.fit_transform(features)

In [8]:
# GRU 입력 형식으로 변환 (samples, time steps, features)
time_steps = 5  # 이전 5일의 데이터를 사용하여 예측

X = []
y = []

for i in range(time_steps, len(scaled_features)):
    X.append(scaled_features[i-time_steps:i])
    y.append(target.values[i])

X, y = np.array(X), np.array(y)

In [9]:
split_ratio = 0.7  # 훈련 세트 비율
split_index = int(split_ratio * len(X))

X_train, y_train = X[:split_index], y[:split_index]
X_test, y_test = X[split_index:], y[split_index:]

In [10]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import GRU, Dense, Dropout

model = Sequential()
model.add(GRU(units=50, return_sequences=True, input_shape=(X.shape[1], X.shape[2])))
model.add(Dropout(0.1))
model.add(GRU(units=20))
model.add(Dropout(0.1))
model.add(Dense(1, activation='sigmoid'))

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [11]:
model.fit(X_train, y_train, epochs=100, batch_size=32, validation_split=0.2)

Epoch 1/100


31/31 [==============================] - 4s 30ms/step - loss: 0.6905 - accuracy: 0.5474 - val_loss: 0.7006 - val_accuracy: 0.4856
Epoch 2/100
31/31 [==============================] - 0s 6ms/step - loss: 0.6880 - accuracy: 0.5320 - val_loss: 0.7143 - val_accuracy: 0.4856
Epoch 3/100
31/31 [==============================] - 0s 6ms/step - loss: 0.6869 - accuracy: 0.5485 - val_loss: 0.6976 - val_accuracy: 0.4856
Epoch 4/100
31/31 [==============================] - 0s 6ms/step - loss: 0.6871 - accuracy: 0.5588 - val_loss: 0.6920 - val_accuracy: 0.5802
Epoch 5/100
31/31 [==============================] - 0s 6ms/step - loss: 0.6869 - accuracy: 0.5485 - val_loss: 0.6920 - val_accuracy: 0.5514
Epoch 6/100
31/31 [==============================] - 0s 6ms/step - loss: 0.6864 - accuracy: 0.5330 - val_loss: 0.6916 - val_accuracy: 0.5144
Epoch 7/100
31/31 [==============================] - 0s 6ms/step - loss: 0.6851 - accuracy: 0.5649 - val_loss: 0.6996 - val_accuracy: 0.4815
Epoch 8/10

In [12]:
predictions = model.predict(X_test)
predictions = (predictions > 0.5).astype(int)  # 이진 분류 문제이므로 0.5를 기준으로 예측

17/17 [==============================] - 1s 2ms/step


In [13]:
from sklearn.metrics import accuracy_score

accuracy = accuracy_score(y_test, predictions)
print(f"Accuracy: {accuracy}")

Accuracy: 0.508637236084453


In [15]:
# 피쳐 중요도 값을 딕셔너리 형태로 정의 (이미 제공된 값 사용)
feature_importance = {
   'wti': 0.221272,
    'dow': 0.175300,
    'usd_krw': 0.081070,
    'gold': 0.057840,
    'vix': 0.024542,
    'jpy_krw': -0.048976,
    'brent': -0.076060,
    'nikkei': -0.109504,
    'kosdaq': -0.119270,
    'cny_krw': -0.132166,
    'ssec': -0.193056
}


# 피쳐 중요도 값을 배열 형태로 정의
feature_importance_arr = np.array(list(feature_importance.values()))

# 피쳐 가중치 적용
for feature, importance in feature_importance.items():
    if feature in df.columns:
        df[feature] *= importance

df.head()

,Unnamed: 0.1,날짜,Unnamed: 0,kospi200,kosdaq,dow,nikkei,ssec,usd_krw,cny_krw,jpy_krw,콜금리,국고채,wti,brent,gold,vix,Target
0,0.0,2017-04-17,1738.0,278.10,-75.076887,3617.652076,-2009.974391,-622.059252,91.853931,-21.758489,-0.509532,1.23,3.611,11.667673,-4.210682,74.723496,0.359786,1.0
1,1.0,2017-04-18,1737.0,278.23,-75.704247,3617.652076,-2016.909279,-617.144046,91.853931,-21.943521,-0.516138,1.23,3.611,11.667673,-4.210682,74.723496,0.359786,0.0
2,2.0,2017-04-19,1736.0,276.49,-75.854527,3597.730984,-2018.399629,-612.120729,92.633825,-21.946164,-0.514522,1.23,3.611,11.579164,-4.174933,74.850744,0.353896,1.0
3,3.0,2017-04-20,1735.0,277.76,-75.831866,3576.907097,-2018.212377,-612.392938,92.718948,-21.856291,-0.509879,1.24,3.611,11.200789,-4.025856,74.231856,0.366412,1.0
4,4.0,2017-04-21,1734.0,280.05,-75.731679,3607.447863,-2039.046608,-612.595646,92.267388,-21.776992,-0.509390,1.24,3.611,11.123343,-4.030419,74.254992,0.347269,1.0


In [16]:
df=df.drop(['Unnamed: 0.1'], axis=1)
df.head()

,날짜,Unnamed: 0,kospi200,kosdaq,dow,nikkei,ssec,usd_krw,cny_krw,jpy_krw,콜금리,국고채,wti,brent,gold,vix,Target
0,2017-04-17,1738.0,278.10,-75.076887,3617.652076,-2009.974391,-622.059252,91.853931,-21.758489,-0.509532,1.23,3.611,11.667673,-4.210682,74.723496,0.359786,1.0
1,2017-04-18,1737.0,278.23,-75.704247,3617.652076,-2016.909279,-617.144046,91.853931,-21.943521,-0.516138,1.23,3.611,11.667673,-4.210682,74.723496,0.359786,0.0
2,2017-04-19,1736.0,276.49,-75.854527,3597.730984,-2018.399629,-612.120729,92.633825,-21.946164,-0.514522,1.23,3.611,11.579164,-4.174933,74.850744,0.353896,1.0
3,2017-04-20,1735.0,277.76,-75.831866,3576.907097,-2018.212377,-612.392938,92.718948,-21.856291,-0.509879,1.24,3.611,11.200789,-4.025856,74.231856,0.366412,1.0
4,2017-04-21,1734.0,280.05,-75.731679,3607.447863,-2039.046608,-612.595646,92.267388,-21.776992,-0.509390,1.24,3.611,11.123343,-4.030419,74.254992,0.347269,1.0


In [17]:
df=df.drop(['Unnamed: 0'], axis=1)
df.head()

,날짜,kospi200,kosdaq,dow,nikkei,ssec,usd_krw,cny_krw,jpy_krw,콜금리,국고채,wti,brent,gold,vix,Target
0,2017-04-17,278.10,-75.076887,3617.652076,-2009.974391,-622.059252,91.853931,-21.758489,-0.509532,1.23,3.611,11.667673,-4.210682,74.723496,0.359786,1.0
1,2017-04-18,278.23,-75.704247,3617.652076,-2016.909279,-617.144046,91.853931,-21.943521,-0.516138,1.23,3.611,11.667673,-4.210682,74.723496,0.359786,0.0
2,2017-04-19,276.49,-75.854527,3597.730984,-2018.399629,-612.120729,92.633825,-21.946164,-0.514522,1.23,3.611,11.579164,-4.174933,74.850744,0.353896,1.0
3,2017-04-20,277.76,-75.831866,3576.907097,-2018.212377,-612.392938,92.718948,-21.856291,-0.509879,1.24,3.611,11.200789,-4.025856,74.231856,0.366412,1.0
4,2017-04-21,280.05,-75.731679,3607.447863,-2039.046608,-612.595646,92.267388,-21.776992,-0.509390,1.24,3.611,11.123343,-4.030419,74.254992,0.347269,1.0


In [18]:
df=df.drop(['국고채'], axis=1)
df.head()

,날짜,kospi200,kosdaq,dow,nikkei,ssec,usd_krw,cny_krw,jpy_krw,콜금리,wti,brent,gold,vix,Target
0,2017-04-17,278.10,-75.076887,3617.652076,-2009.974391,-622.059252,91.853931,-21.758489,-0.509532,1.23,11.667673,-4.210682,74.723496,0.359786,1.0
1,2017-04-18,278.23,-75.704247,3617.652076,-2016.909279,-617.144046,91.853931,-21.943521,-0.516138,1.23,11.667673,-4.210682,74.723496,0.359786,0.0
2,2017-04-19,276.49,-75.854527,3597.730984,-2018.399629,-612.120729,92.633825,-21.946164,-0.514522,1.23,11.579164,-4.174933,74.850744,0.353896,1.0
3,2017-04-20,277.76,-75.831866,3576.907097,-2018.212377,-612.392938,92.718948,-21.856291,-0.509879,1.24,11.200789,-4.025856,74.231856,0.366412,1.0
4,2017-04-21,280.05,-75.731679,3607.447863,-2039.046608,-612.595646,92.267388,-21.776992,-0.509390,1.24,11.123343,-4.030419,74.254992,0.347269,1.0


In [19]:
# 쉼표 제거 및 숫자 형식으로 변환, 날짜 열은 제외
for column in df.columns:
    if column != '날짜':
        df[column] = df[column].apply(lambda x: str(x).replace(',', '') if isinstance(x, str) else x)
        df[column] = df[column].astype(float)

In [20]:
# 날짜 칼럼을 datetime 객체로 변환
df['날짜'] = pd.to_datetime(df['날짜'])
df.set_index('날짜', inplace=True)

In [21]:
# 피처와 타겟을 분리합니다
features = df.drop('Target', axis=1)
target = df['Target']

# 피처 스케일링
scaler = MinMaxScaler()
scaled_features = scaler.fit_transform(features)

# LSTM 입력 형식으로 변환 (samples, time steps, features)
time_steps = 5  # 이전 5일의 데이터를 사용하여 예측

X = []
y = []

for i in range(time_steps, len(scaled_features)):
    X.append(scaled_features[i-time_steps:i])
    y.append(target.values[i])

X, y = np.array(X), np.array(y)

In [22]:
split_ratio = 0.7  # 훈련 세트 비율
split_index = int(split_ratio * len(X))

X_train, y_train = X[:split_index], y[:split_index]
X_test, y_test = X[split_index:], y[split_index:]

In [23]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import GRU, Dense, Dropout

model = Sequential()
model.add(GRU(units=50, return_sequences=True, input_shape=(X.shape[1], X.shape[2])))
model.add(Dropout(0.1))
model.add(GRU(units=20))
model.add(Dropout(0.1))
model.add(Dense(1, activation='sigmoid'))

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [24]:
model.fit(X_train, y_train, epochs=100, batch_size=32, validation_split=0.2)

Epoch 1/100
31/31 [==============================] - 3s 25ms/step - loss: 0.6877 - accuracy: 0.5371 - val_loss: 0.6918 - val_accuracy: 0.5638
Epoch 2/100
31/31 [==============================] - 0s 6ms/step - loss: 0.6835 - accuracy: 0.5515 - val_loss: 0.6916 - val_accuracy: 0.5597
Epoch 3/100
31/31 [==============================] - 0s 6ms/step - loss: 0.6911 - accuracy: 0.5134 - val_loss: 0.6914 - val_accuracy: 0.5391
Epoch 4/100
31/31 [==============================] - 0s 8ms/step - loss: 0.6843 - accuracy: 0.5608 - val_loss: 0.6942 - val_accuracy: 0.4815
Epoch 5/100
31/31 [==============================] - 0s 6ms/step - loss: 0.6861 - accuracy: 0.5330 - val_loss: 0.6919 - val_accuracy: 0.5144
Epoch 6/100
31/31 [==============================] - 0s 6ms/step - loss: 0.6844 - accuracy: 0.5505 - val_loss: 0.7017 - val_accuracy: 0.4774
Epoch 7/100
31/31 [==============================] - 0s 6ms/step - loss: 0.6819 - accuracy: 0.5629 - val_loss: 0.6925 - val_accuracy: 0.5144
Epoch 8/100


In [25]:
predictions = model.predict(X_test)
predictions = (predictions > 0.5).astype(int)  # 이진 분류 문제이므로 0.5를 기준으로 예측

17/17 [==============================] - 1s 2ms/step


In [26]:
from sklearn.metrics import accuracy_score

accuracy = accuracy_score(y_test, predictions)
print(f"Accuracy: {accuracy}")


Accuracy: 0.508637236084453
